# Проект: Exploratory Data Analysis and Feature Engineering #

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Для корректной работы Jupyter Notebook
%matplotlib inline
# Для корректного отображения графиков в тёмной теме
plt.style.use('default')